In [11]:
import numpy as np
import scipy.stats as scistats
N = 11
K = 1

sig_ub = 0.00025693997315943056 * (N)/(N - K - 1)

Sigma = np.array([[0.46632500232639523, -0.013567303132596577],
                  [-0.013567303132596577, 0.0004903140988266602]])

print(sig_ub * Sigma)

[[ 1.46443652e-04 -4.26064528e-06]
 [-4.26064528e-06  1.53977134e-07]]


In [19]:
def feature_transform(X, K):
    
    return X**np.arange(K + 1)

def f_x(X, omega):
    
    K = omega.shape[0] - 1 #Index starts at zero
    
    Xtransform = feature_transform(X, K)
    
    return np.dot(Xtransform, omega)

def log_likelihood_function(X, t, omega, sigma):
    
    term1 = -1/2 * np.log(2 * np.pi)
    term2 = -1/2 * np.log(sigma**2)
    
    t_pred = f_x(X, omega)
    
    term3 = -1/(2 * sigma**2) * np.sum((t - t_pred)**2, axis = 0)
    
    log_likelihood = term1 + term2 + term3
    
    return log_likelihood

def Phi_func(X, K):
    
    return X**np.arange(K + 1)

def maximum_likelihood_solution(X, t, K, biased = True, verbose = False):

    #calculate Phi
    Phi = Phi_func(X, K)

    #calculate ATA
    ATA = np.dot(Phi.T, Phi)

    #Calculate ATb
    ATb = np.dot(Phi.T, t)

    #Check shapes are K + 1 x K + 1 and K + 1 x 1
    #print(ATA.shape, ATb.shape)

    #Calculate Omega
    omega_ML = np.linalg.solve(ATA, ATb)

    #Calculate sigma
    t_pred = f_x(X, omega_ML)

    if biased:
        divisor = X.shape[0]
    
    else:
        divisor = X.shape[0] - K - 1
    
    sigma_ML = np.sqrt(1/divisor * np.sum((t_pred - t)**2, axis = 0))

    #Calculate the log-likelihood:
    LL = log_likelihood_function(X, t, omega_ML, sigma_ML)[0]

    if verbose: 
        print("For a {}th order polynomial:".format(K))
        print("\nOptimal weights:\n{}\n".format(omega_ML))
        print("Optimal sigma:\n{}\n".format(sigma_ML))
        print("Optimal model log-likelihood:\n{}\n".format(LL))
    
    return omega_ML, sigma_ML, LL

def confidence_interval(parameters, X, gamma = 0.9, weight_index = 0):
    
    #Define the student-t distribution
    pdf_t = scistats.t
    N = X.shape[0]
    K = parameters[0].shape[0] - 1
    print(N, K)
    dof = N - K - 1
    
    #Unbiased variance
    mean = parameters[0][weight_index]
    sigma_unbiased = parameters[1]**2 * (N)/(N - K - 1) #parameter[1] is the standard deviation
    
    Phi = Phi_func(X, K)
    
    A = np.linalg.inv(np.dot(Phi.T, Phi))
    
    Sigma = sigma_unbiased * A
    
    print(Sigma, (1 - gamma)/2)

    lower_bound = pdf_t.ppf(q = (1 - gamma)/2, df = dof, loc = mean, scale = np.sqrt(Sigma[weight_index, weight_index]))
    upper_bound = pdf_t.ppf(q = (1 + gamma)/2, df = dof, loc = mean, scale = np.sqrt(Sigma[weight_index, weight_index]))
    
    return lower_bound, upper_bound

x = np.array([[5.133, 10.124, 15.060, 19.946, 24.899, 29.792, 29.877, 35.011, 39.878, 44.862, 49.795]]).T
y = np.array([[0.265,0.287,0.282,0.286,0.310,0.333, 0.343,0.335,0.311, 0.345,0.319]]).T

omega = np.array([[0.26883427, 0.00150742]]).T

omega_ML, sigma_ML, LL = maximum_likelihood_solution(x, 
                                                     y, 
                                                     K = 1, 
                                                     biased = True, 
                                                     verbose = False)

print(omega_ML)

confidence_interval((omega_ML, sigma_ML), x, gamma = 0.95, weight_index = 0)

[[0.26883427]
 [0.00150742]]
11 1
[[ 1.46443652e-04 -4.26064528e-06]
 [-4.26064528e-06  1.53977134e-07]] 0.025000000000000022


(array([0.24145902]), array([0.29620952]))